In [1]:
import fastai
from fastai import *
from fastai.text import * 
from fastai.text.models import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score

import nltk
from nltk.corpus import stopwords
import re 
from bs4 import BeautifulSoup

from nltk.stem.snowball import SnowballStemmer
from pandarallel import pandarallel

In [2]:
l = load_learner('.','export.pkl')

In [3]:
olddata = pd.read_pickle('newdata.pkl')

In [4]:
newdata_eng = olddata[olddata['Language']== 'eng']

In [6]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [7]:
#PREPROCEESING THE TEXT

def text_preprocessing_simple(text):
    # remove non-letters
    text_non_letters = re.sub("[^a-zA-Z']", " ", text) 
    # convert to lower-case
    text_lower = text_non_letters.lower()
    return text_lower

In [8]:
def get_preds_confidence(text):
    preds = l.predict(text)
    pred_class = str(preds[0])
    pred_max = preds[2].max().item()
    return (pred_class, pred_max)

In [9]:
newdata_eng['Text'] = newdata_eng['Text'].apply(lambda x: text_preprocessing_simple(x))

/home/arul/anaconda3/envs/ath/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
from tqdm.notebook import trange, tqdm
tqdm.pandas()

/home/arul/anaconda3/envs/ath/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [12]:
newdata_eng['pred']= newdata_eng['Text'].progress_apply(get_preds_confidence)

/home/arul/anaconda3/envs/ath/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
newdata_eng['confidence'] = newdata_eng['pred'].apply(lambda x: x[1])

/home/arul/anaconda3/envs/ath/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
newdata_eng['pred'] = newdata_eng['pred'].apply(lambda x: x[0])

/home/arul/anaconda3/envs/ath/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
conf_90 = newdata_eng[newdata_eng['confidence'] < 0.90]

In [26]:
len(conf_90), (len(conf_90)/len(newdata_eng))*100

(498, 10.101419878296145)

In [18]:
pd.crosstab(conf_90.Sentiment, conf_90.pred)

pred,-1,0,1
Sentiment,,,
-1,123,36,41
0,25,44,50
1,36,29,114


In [19]:
conf_80 = newdata_eng[newdata_eng['confidence'] < 0.80]

In [24]:
len(conf_80), (len(conf_80)/len(newdata_eng))*100

(327, 6.632860040567952)

In [20]:
pd.crosstab(conf_80.Sentiment, conf_80.pred)

pred,-1,0,1
Sentiment,,,
-1,77,25,33
0,20,32,26
1,26,17,71


In [29]:
newdata_eng.rename(columns={'pred': 'Predicted Sentiment', 'confidence': 'Confidence Level', 'Text': 'Processed Text', 'raw_text':'Raw Text'}, inplace=True)

/home/arul/anaconda3/envs/ath/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [30]:
newdata_eng.to_csv('Predictions with Confidence.csv')